This file preserve the SQL query I used for task 1. 

### Q1: How many bike trips have been started in San Jose?

In [ ]:
#standardSQL
SELECT
start_station_name, lon, lat, COUNT(trip_id) AS cnt
FROM
  `bigquery-public-data.san_francisco_bikeshare.bikeshare_trips` AS trip
INNER JOIN 
  (SELECT station_id, region_id, lon, lat
  FROM `bigquery-public-data.san_francisco_bikeshare.bikeshare_station_info` AS S
  INNER JOIN 
    (SELECT region_id AS id
    FROM`bigquery-public-data.san_francisco_bikeshare.bikeshare_regions`
    WHERE name = "San Jose")AS R
  ON S.region_id = R.id
    ) AS station
ON trip.start_station_id =station.station_id
GROUP BY start_station_name, lon, lat



### Q2: Which bike station has the oldest members in average?

### Q3: During which month in 2018 did customers ride the longest per trip?

### Q4: What are the top 3 starting stations for customers with a subscription?

### Q5: What is the average time difference between start dates of bike trips for the station “SAP Center”?